In [1]:
# Import required libraries
import pandas as pd
!pip install dash
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 19.1 MB/s eta 0:00:00


In [3]:
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')

app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.H1('SpaceX Launch Data Analysis', style={'textAlign': 'center'}),

    html.Div([
        html.Label('Select a Launch Site:'),
        dcc.Dropdown(id='site-dropdown',
                     options=[
                         {'label': 'All Sites', 'value': 'ALL'},
                         {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                         {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                         {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                         {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                     ],
                     value='ALL',
                     placeholder="Select a Launch Site here",
                     searchable=True
                     ),
        html.Br(),
        html.Label('Select Payload Range (kg):'),
        dcc.RangeSlider(id='payload-slider',
                        min=0, max=10000, step=1000,
                        marks={0: '0', 10000: '10000'},
                        value=[df['Payload Mass (kg)'].min(), df['Payload Mass (kg)'].max()])
    ]),

    html.Br(),
    html.Br(),

    html.Div([
        html.Div(dcc.Graph(id='success-pie-chart')),
        html.Div(dcc.Graph(id='success-payload-scatter-chart'))
    ], style={'display': 'flex'}),
])

@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(df, values=df.groupby('Launch Site')['class'].sum(), names=df['Launch Site'].unique(), title='Total Success Launches by Site')
        return fig
    else:
        filtered_df = df[df['Launch Site'] == entered_site]
        fig = px.pie(filtered_df, names='class', title=f'Success vs Failure at {entered_site}')
        return fig

@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id="payload-slider", component_property="value")])
def get_scatter_chart(selected_site, selected_payload):
    if selected_site == 'ALL':
        filtered_df = df[(df['Payload Mass (kg)'] >= selected_payload[0]) & (df['Payload Mass (kg)'] <= selected_payload[1])]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category', title='Payload vs Success Rate (All Sites)')
        return fig
    else:
        filtered_df = df[(df['Launch Site'] == selected_site) & (df['Payload Mass (kg)'] >= selected_payload[0]) & (df['Payload Mass (kg)'] <= selected_payload[1])]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category', title=f'Payload vs Success Rate at {selected_site}')
        return fig

if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>